### Metaprogramming - Application 1

In [1]:
class Point2D:
    __slots__ = ('_x', '_y')

    def __init__(self, x, y):
        self._x = x
        self._y = y

    @property
    def x(self):
        return self._x

    @property
    def y(self):
        return self._y

    def __eq__(self, other):
        return isinstance(other, Point2D) and (self._x, self._y)  == (other.x, other.y)

    def __hash__(self):
        return hash((self.x, self.y))

    def __repr__(self):
        return f'Point2D({self.x}, {self.y})'

    def __str__(self):
        return f'({self.x}, {self.y})'


class Point3D:
    __slots__ = ('_x', '_y', '_z')

    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

    @property
    def x(self):
        return self._x

    @property
    def y(self):
        return self._y

    @property
    def z(self):
        return self._z

    def __eq__(self, other):
        return isinstance(other, Point3D) and (self.x, self.y, self.z) == (other.x, other.y, other.z)

    def __hash__(self):
        return hash((self.x, self.y, self.z))

    def __repr__(self):
        return f'Point2D({self.x}, {self.y}, {self.z})'

    def __str__(self):
        return f'({self.x}, {self.y}, {self.z})'


In [2]:
class Point2D:
    _fields = ['x', 'y']

    def __init__(self, x, y):
        self._x = x
        self._y = y

class Point3D:
    _fields = ['x', 'y', 'z']

    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

In [3]:
class SlottedStruct(type):
    def __new__(mcls, name, bases, class_dict):
        cls_object = super().__new__(mcls, name, bases, class_dict)

        # __slots__
        setattr(cls_object, '__slots__', [f'_{field}' for field in cls_object._fields])

        # read-only properties
        for field in cls_object._fields:
            slot = f'_{field}'
            setattr(cls_object, field, property(fget=lambda self, attrib=slot: getattr(self, attrib)))

        return cls_object

In [4]:
class Person(metaclass=SlottedStruct):
    _fields = ['name', 'age']

    def __init__(self, name, age):
        self._name = name
        self._age = age

In [5]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '_fields': ['name', 'age'],
              '__init__': <function __main__.Person.__init__(self, name, age)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              '__slots__': ['_name', '_age'],
              'name': <property at 0x7f5f717a0cc0>,
              'age': <property at 0x7f5f717a0770>})

In [6]:
p = Person('Alex', 19)

In [7]:
p.name

'Alex'

In [8]:
p.name = 'Guido'

AttributeError: property of 'Person' object has no setter

In [20]:
class SlottedStruct(type):
    def __new__(mcls, name, bases, class_dict):
        cls_object = super().__new__(mcls, name, bases, class_dict)

        # __slots__
        setattr(cls_object, '__slots__', [f'_{field}' for field in cls_object._fields])

        # read-only properties
        for field in cls_object._fields:
            slot = f'_{field}'
            setattr(cls_object, field, property(fget=lambda self, attrib=slot: getattr(self, attrib)))

        # __eq__
        def eq(self, other):
            if isinstance(other, cls_object):
                self_fields = [getattr(self, field) for field in cls_object._fields]
                other_fields  = [getattr(other, field) for field in cls_object._fields]
                return self_fields == other_fields
            return False
        setattr(cls_object, '__eq__', eq)

        # __hash__
        def hash_(self):
            field_values = (getattr(self, field) for field in cls_object._fields)
            return hash(tuple(field_values))
        setattr(cls_object, '__hash__', hash_)

        # __str__
        def str_(self):
            field_values = (getattr(self, field) for field in cls_object._fields)
            field_values_joined = ', '.join(map(str, field_values))
            return f'{cls_object.__name__}({field_values_joined})'
        setattr(cls_object, '__str__', str_)

        # __repr__
        # all the above are closures, this one uses 'self' instead of 'cls_object' and is not a closure
        # just as an example that it still work fine
        def repr_(self):
            field_values = (getattr(self, field) for field in self._fields)
            field_key_values = (f'{key}={value}' for key, value in zip(self._fields, field_values))
            field_key_values_str = ', '.join(field_key_values)
            return f'{self.__class__.__name__}({field_key_values_str})'
        setattr(cls_object, '__repr__', repr_)


        return cls_object

In [21]:
class Person(metaclass=SlottedStruct):
    _fields = ['name']

    def __init__(self, name):
        self._name = name

In [22]:
type(Person)

__main__.SlottedStruct

In [23]:
p1 = Person('Alex')

In [24]:
p2 = Person('Alex')

In [25]:
type(p1), isinstance(p1, Person)

(__main__.Person, True)

In [26]:
p1 == p2

True

In [27]:
hash(p1), hash(p2)

(6558935243865745459, 6558935243865745459)

In [28]:
str(p1)

'Person(Alex)'

In [29]:
repr(p1)

'Person(name=Alex)'

In [30]:
class Point2D(metaclass=SlottedStruct):
    _fields = ['x', 'y']

    def __init__(self, x, y):
        self._x = x
        self._y = y

class Point3D(metaclass=SlottedStruct):
    _fields = ['x', 'y', 'z']

    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

In [31]:
p1 = Point2D(1, 2)
p2 = Point2D(1, 2)
p3 = Point2D(0, 0)

In [33]:
repr(p1), str(p1), repr(p2), str(p2), p1.x, p2.y

('Point2D(x=1, y=2)',
 'Point2D(1, 2)',
 'Point2D(x=1, y=2)',
 'Point2D(1, 2)',
 1,
 2)